In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import humanize
import plotly.express as px

# Data For Good - Tax Observatory Dataviz

Visualizations for the EU Tax Observatory based on the [EU Tax Obs - Dataviz - Viz](https://docs.google.com/document/d/1q_7x8NrEWNsak9LSE3TxIiSphhVcqWqn6YeG1kQNQTk/edit#heading=h.97g77izer615) Google Docs.

## Table of contents
* [Initialization](#initialization)
* [Utils](#utils)
* [#12 - available reports by company](#12---available-reports-by-company)
* [#13 - company key financials kpis](#13---company-key-financials-kpis)
* [#14 - company top jurisdictions for revenue](#14---company-top-jurisdictions-for-revenue)
* [#15 - company’s % pre-tax profit and % employees by jurisdiction](#15---companys--pre-tax-profit-and--employees-by-jurisdiction)
* [#18 - breakdown of revenue between related party and unrelated party in TH vs domestic vs non TH](#18---breakdown-of-revenue-between-related-party-and-unrelated-party-in-th-vs-domestic-vs-non-th)
* [#21 - evolution of tax havens use over time : % profit vs % employees in TH over time](#21---evolution-of-tax-havens-use-over-time---profit-vs--employees-in-th-over-time)
* [#22 - locations of profits booked vs. mean 3Y ETR](#22---locations-of-profits-booked-vs-mean-3y-etr)

## Initialization

In [35]:
# Import dataset in a DataFrame
df = pd.read_csv('data_final_dataviz.csv', sep=',')

In the website it will be possible to select a specific company. To work on the viz creation, we simulate the selection by picking a random company but also a random year.

In [36]:
# Select a random company and a random year
def pick_random_company_and_year() -> tuple:
    company = np.random.choice(df['mnc'].unique())
    print(f'Selected company : {company}.')

    year = np.random.choice(df.loc[df['mnc'] == company, 'year'].unique())
    print(f'Selected year : {year}.')

    return company, year

In [37]:
company, year = pick_random_company_and_year()
# company, year = 'SHELL', 2020

Selected company : RIO TINTO.
Selected year : 2020.


## Utils

In [38]:
def compute_missing_values(df: pd.DataFrame=df):
    df = df.loc[df['year'] == 2022, ['mnc', 'year', 'profit_before_tax', 'employees']]

    df['both'] = np.where(
        df['profit_before_tax'].isna() & df['employees'].isna(),
        np.nan,
        9999
    )

    mnc_list = df['mnc'].unique()

    data = pd.DataFrame()

    for mnc in mnc_list:
        n_rows = len(df.loc[df['mnc'] == mnc])
        data[mnc] = df.loc[
            df['mnc'] == mnc, ['profit_before_tax', 'employees', 'both']
        ].isna().sum() / n_rows

    return data.T

In [39]:
# df_na = compute_missing_values()
# df_na.to_csv('na2022.csv')

## #12 - available reports by company

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|tbd (goal is on company page, to show reports tracked, so could be just a table showing year available for a company)|-||`company explo` need a filter by company||

In [40]:
def compute_company_available_reports(df: pd.DataFrame, company: str) -> dict:
    """Compute the number of reports tracked for a specific company and the 
    available fiscal years.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): company name.

    Returns:
        dict: numbers of reports and fiscal years.
    """
    available_years = df.loc[df['mnc'] == company, 'year'].unique()
    n_reports = len(available_years)

    # Convert type of items from 'int' to 'str' in available years list
    years_string_list = [str(year) for year in available_years]
    
    # Summarize all available years in one string
    if len(years_string_list) == 1:
        years_string = years_string_list[0]
    elif len(years_string_list) > 1:
        years_string = ', '.join(years_string_list[:-1])
        years_string += ' and ' + years_string_list[-1]

    # Create a dictionnary with the results
    data = {
        'Company': company,
        'Reports': n_reports,
        'Fiscal year(s) available': years_string
    }

    return data


def display_company_available_reports(
        df: pd.DataFrame, company: str, hide_company: bool=True) -> pd.DataFrame:
    """Display the number of reports tracked for a specific company and the 
    available fiscal years.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): company name.
        hide_company (bool, optional): hide company name in final table. Defaults to True.

    Returns:
        pd.DataFrame: numbers of reports and fiscal years.
    """
    
    # Compute data
    data = compute_company_available_reports(df=df, company=company)

    # Create the table
    df = pd.DataFrame.from_dict(data=data, orient='index')

    if hide_company:
        return df[1:].style.hide(axis='columns')

    return df.style.hide(axis='columns')

In [41]:
display_company_available_reports(df=df, company=company, hide_company=True)

Reports,4
Fiscal year(s) available,"2018, 2019, 2020 and 2021"


## #13 - company key financials kpis

**kpis** : hq country, total revenue, total unrelated party revenues, total related party revenues, pre-tax profits, taxes paid, employees.

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|raw figure for each, or table with indicator name in col A, value in col B|hq country : -<br><br>metrics : sum (metric) by year by mnc|**giulia : keep all values, even negative values**|`company explo` need a filter by company||

In [42]:
def compute_company_key_financials_kpis(
        df: pd.DataFrame, company: str, year: int=None) -> dict:
    """Compute key financial KPIs for a company.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int, optional): fiscal year to filter the results with. Defaults to None.

    Returns:
        dict: company key financial KPIs.
    """
    
    kpis_list = ['total_revenues', 'unrelated_revenues', 'related_revenues', 
                 'profit_before_tax', 'tax_paid', 'employees']
    
    years_list = df.loc[df['mnc'] == company, 'year'].unique()

    # Compute sum of kpis
    if not year or year not in years_list:
        df = (df.loc[df['mnc'] == company]
            .groupby(['year', 'upe_name'], as_index=False)[kpis_list]
            .sum()
        )
    else:
        df = (df.loc[(df['mnc'] == company) & (df['year'] == year)]
              .groupby(['year', 'upe_name'], as_index=False)[kpis_list]
              .sum())

    df = df.set_index('year')

    # Make financial numbers easily readable with 'humanize' package
    for column in df.columns:
        if column not in ['employees', 'upe_name']:
            df[column] = df[column].apply(
                lambda x: humanize.intword(x) if isinstance(x, (int, float)) else x)
            df[column] = '€ ' + df[column]
        elif column == 'employees':
            df[column] = df[column].astype(int)

    # Clean columns string
    df = df.rename(columns={'upe_name': 'headquarter'})
    df.columns = df.columns.str.replace('_', ' ').str.capitalize()

    # Create a dictionnary with the results
    data = df.to_dict(orient='index')

    return data


def display_company_key_financials_kpis(
        df: pd.DataFrame, company: str, year: int=None) -> pd.DataFrame:
    """Display key financial KPIs for a company.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int, optional): fiscal year to filter the results with. Defaults to None.

    Returns:
        pd.DataFrame: company key financial KPIs.
    """
    
    # Compute data
    data = compute_company_key_financials_kpis(df=df, company=company, year=year)

    # Create the table
    df = pd.DataFrame.from_dict(data)

    return df

In [43]:
display_company_key_financials_kpis(df=df, company=company)

,2018,2019,2020,2021
Headquarter,Australia,Australia,Australia,Australia
Total revenues,€ 56.6 billion,€ 55.7 billion,€ 53.5 billion,€ 75.4 billion
Unrelated revenues,€ 39.4 billion,€ 39.6 billion,€ 40.1 billion,€ 54.5 billion
Related revenues,€ 17.1 billion,€ 16.1 billion,€ 13.4 billion,€ 20.9 billion
Profit before tax,€ 15.1 billion,€ 9.7 billion,€ 13.2 billion,€ 25.2 billion
Tax paid,€ 2.9 billion,€ 4.1 billion,€ 4.6 billion,€ 7.2 billion
Employees,44413,42903,44309,46092


## #14 - company top jurisdictions for revenue

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|horizontal bar chart showing % total revenue for top 9 jurisdictions + rest as “OTHERS”|sum(revenue) by company x hq country x year, display top 9 and group remaining countries||`company explo` need a filter by company||

In [44]:
def compute_top_jurisdictions_revenue(
    df: pd.DataFrame, company: str, year: int) -> dict:
    """Rank jurisdictions on their percentage of total revenues in a top 10.
    When there are more than 10 jurisdictions, the tenth represent all 
    jurisdictions below 9.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year.

    Returns:
        dict: Top 10 jurisdictions for percentage of total revenues.
    """
    
    df = df.loc[
        (df['mnc'] == company) & (df['year'] == year), 
        ['jur_name', 'related_revenues', 'unrelated_revenues', 'total_revenues']
    ]

    # Calculate missing values in 'total_revenues' if 'related_revenues' and 
    # 'unrelated_revenues' are available
    df.loc[
        df['related_revenues'].notna() 
        & df['unrelated_revenues'].notna()
        & df['total_revenues'].isna(),
        'total_revenues'
    ] = df['related_revenues'] + df['unrelated_revenues']

    # Subset DataFrame
    df = df[['jur_name', 'total_revenues']]

    # Remove rows where 'total_revenues' is missing
    df = df.dropna(subset=['total_revenues'])

    # Group same 'jur_name' (sometimes several 'Other')
    # e.g. SWISS LIFE, 2021
    df = df.groupby('jur_name', as_index=False).sum()

    # Filter the top 10 'jur_name' for 'total_revenues'
    if len(df) > 10:
        # Check if 'Other' already in 'jur_name' and add the revenues
        # of the 'jur_name' below top 10 to its value
        if 'Other' in df['jur_name'].values:
            top = df.nlargest(10, 'total_revenues')
            below_top_revenues = df.loc[
                ~df['jur_name'].isin(top['jur_name']), 'total_revenues'].sum()
            top.loc[top['jur_name'] == 'Other', 'total_revenues'] += below_top_revenues
            top = top.reset_index(drop=True)
        else:
            # Keep top 9 and group all revenues of the rest in 'Others'
            top = df.nlargest(9, 'total_revenues')
            below_top_revenues = df.loc[
                ~df['jur_name'].isin(top['jur_name']), 'total_revenues'].sum()
            top = top.reset_index(drop=True)
            top.loc[9] = ['Others', below_top_revenues]
    else:
        top = df
    
    # Rename 'Other' to 'Others'
    top.loc[top['jur_name'] == 'Other', 'jur_name'] = 'Others'

    # Compute percentage of revenue
    top['total_revenues_%'] = top['total_revenues'] / top['total_revenues'].sum()
  
    # Convert DataFrame to dictionnary
    data = top.to_dict()
    
    return data


def display_jurisdictions_top_revenue(df: pd.DataFrame, company: str, year: int):
    """Display top 10 jurisdictions for percentage of total revenues in an
    horizontal bar chart.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year.
    """
    
    # Compute data
    data = compute_top_jurisdictions_revenue(df=df, company=company, year=year)

    # Create DataFrame
    df = pd.DataFrame.from_dict(data)
    df = df.sort_values(by='total_revenues_%')
    
    # Create figure
    fig = px.bar(df, x='total_revenues_%', y='jur_name',
                  orientation='h', title='Top jurisdictions for revenue', text_auto='.1%')

    # Update layout settings
    fig.update_layout(
        xaxis=dict(
            title='Percentage of total revenue',
            tickformat='.0%'
        ),
        yaxis_title=None,
        plot_bgcolor='white',
        width=800,
        height=480
    )

    # Define position of text values
    values_positions = [
        'outside' if value <= 0.05 else 'inside' for value in df['total_revenues_%']]

    fig.update_traces(
        textangle=0,
        textposition=values_positions,
        selector=dict(name='')
    )

    # Define style of hover on bars
    fig.update_traces(hovertemplate='%{y}: %{x: .3%}')

    fig.show()

In [45]:
display_jurisdictions_top_revenue(df=df, company=company, year=year)

## #15 - company’s % pre-tax profit and % employees by jurisdiction

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|horizontal bar chart with one bar for % pre tax profit, the other for % employees, in different colors rank by % pre-tax profit descending|for a company : filter (profit >0) profits by JUR / total profit vs employees by JUR / total employees|**giulia : keep only positive values**|`company explo` need a filter by company||

In [46]:
def compute_pretax_profit_and_employees_rank(
        df: pd.DataFrame, company: str, year: int) -> dict:
    """Compute jurisdictions percentage of profit before tax and percentage
    of employees and rank by percentage of profit.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year.

    Returns:
        dict: rank of jurisdictions with percentage of profit before and percentage
        of employees.
    """
    
    df = df.loc[
        (df['mnc'] == company) & (df['year'] == year) & (df['profit_before_tax'] > 0), 
        ['jur_name', 'profit_before_tax', 'employees']
    ]

    # df = df.fillna(0)
    df = df.sort_values(by='profit_before_tax').reset_index(drop=True)

    df['profit_before_tax_%'] = df['profit_before_tax'] / df['profit_before_tax'].sum()
    df['employees_%'] = df['employees'] / df['employees'].sum()
    df = df.drop(columns=['profit_before_tax', 'employees'])

    data = df.to_dict()

    return data


def display_pretax_profit_and_employees_rank(
        df: pd.DataFrame, company: str, year: int):
    """Display rank of jurisdictions by percentage of profit before and percentage
        of employees.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year.
    """
    
    df = pd.DataFrame(
        compute_pretax_profit_and_employees_rank(df=df, company=company, year=year))

    df = df.rename(
        columns={'profit_before_tax_%': 'Pre-tax profit', 'employees_%': 'Employees'})

    # Create figure
    fig = px.bar(df, x=['Employees', 'Pre-tax profit'], y='jur_name', barmode='group',
                 orientation='h',  title='Jurisdictions pre-tax profit & employees', text_auto='.2%')
    
    fig_height = max(480, (48*len(df['jur_name'])))

    # Set maximum value for xaxis
    if not df[['Pre-tax profit', 'Employees']].isna().all().all():
        max_x_value = max(df[['Pre-tax profit', 'Employees']].max(axis='columns')) + 0.1
    else:
        max_x_value = 1
    
    # Update layout settings
    fig.update_layout(
        xaxis=dict(
            title=None,
            tickformat='.0%',
            range=[0, max_x_value ]
        ),
        yaxis_title=None,
        legend=dict(
            title=dict(text=''),
            orientation='h'
        ),
        plot_bgcolor='white',
        width=800,
        height=fig_height
    )

    # Add annotations for NaN values in 'Employees' and 'Profit Before Tax'
    for index, row in df.iterrows():
        if pd.isna(row['Employees']):
            fig.add_annotation(
                xanchor='left',
                x=0.001,
                y=df.index[index],
                yshift=-10,
                text='Information not provided',
                showarrow=False,
                font=dict(size=12)
            )
        if pd.isna(row['Pre-tax profit']):
            fig.add_annotation(
                xanchor='left',
                x=0.001,
                y=df.index[index],
                yshift=10,
                text='Information not provided',
                showarrow=False,
                font=dict(size=12)
            )

    # Loop through each bar trace and hide the text if the value is NaN
    for trace in fig.data:
        values = df[trace.name]
        text_position = ['outside' if not np.isnan(value) else 'none' for value in values]
        trace.textposition = text_position

    # Define style of hover on bars
    fig.update_traces(hovertemplate='%{y: text}: %{x: .3%}')

    fig.show()

In [47]:
display_pretax_profit_and_employees_rank(df=df, company=company, year=year)

## #18 - breakdown of revenue between related party and unrelated party in TH vs domestic vs non TH

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|horizontal barchart % of revenue from related-party revenue, and % revenue from unrelated party, show value for domestic (hq country) vs tax havens vs. non havens|get total revenue in TH, in domestic country, in non TH then show the % coming from unrelated party revenue and related party revenue (sum of both should equal total revenue)||`company explo` need a filter by company|what if sum or unrelated party revenue  and related party revenue does not equal to total revenue ? safeguard ?|

In [48]:
def compute_related_and_unrelated_revenues_breakdown(
        df: pd.DataFrame, company: str, year: int) -> dict:
    """Compute related and unrelated revenues in tax heaven, non tax heaven and
    domestic jusrisdictions.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year to filter the results with.

    Returns:
        dict: revenues percentage for different type of jurisdictions.
    """
    
    # Create a filtered version of the dataset
    df_temp = df.loc[(df['mnc'] == company) & (df['year'] == year)].copy()

    # Create a boolean column to check if 'jur_code' is the domestic country
    df_temp['domestic'] = df_temp.apply(
        lambda row: row['jur_code'] == row['upe_code'], axis='columns')

    # Calculate missing values in 'related_revenues' if 'unrelated_revenues' and 'total_revenues' are available
    df_temp.loc[
        df_temp['unrelated_revenues'].notna() 
        & df_temp['total_revenues'].notna() 
        & df_temp['related_revenues'].isna(), 
        'related_revenues'
    ] = df_temp['total_revenues'] - df_temp['unrelated_revenues']

    # Calculate missing values in 'unrelated_revenues' if 'related_revenues' and 'total_revenues' are available
    df_temp.loc[
        df_temp['related_revenues'].notna() 
        & df_temp['total_revenues'].notna() 
        & df_temp['unrelated_revenues'].isna(), 
        'unrelated_revenues'
    ] = df_temp['total_revenues'] - df_temp['related_revenues']

    # Remove columns where 'unrelated_revenues' or 'related_revenues' is missing
    df_temp = df_temp.dropna(subset=['unrelated_revenues', 'related_revenues'])

    # Compute kpis in a new DataFrame
    data = pd.DataFrame()
    data['tax_haven'] = df_temp.loc[
        df_temp['jur_tax_haven'] == True, ['unrelated_revenues', 'related_revenues']].sum()
    data['non_tax_haven'] = df_temp.loc[
        df_temp['jur_tax_haven'] == False, ['unrelated_revenues', 'related_revenues']].sum()
    data['domestic'] = df_temp.loc[
        df_temp['domestic'] == True, ['unrelated_revenues', 'related_revenues']].sum()

    # Replace values with share (%) of 'unrelated/related revenues'
    data = data.div(data.sum(axis='rows'), axis='columns')

    # Convert DataFrame to dictionnary
    data = data.to_dict()

    return data


def display_related_and_unrelated_revenues_breakdown(
        df: pd.DataFrame, company: str, year: int):
    """Display related and unrelated revenues in tax heaven, non tax heaven and
    domestic jusrisdictions.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year to filter the results with.
    """
    
    # Compute data
    data = compute_related_and_unrelated_revenues_breakdown(df=df, company=company, year=year)

    # Create DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')

    # Rename columns and indexes
    df.columns = df.columns.str.replace('_', ' ').str.capitalize()
    df.index = df.index.str.replace('_', ' ').str.capitalize()
    
    # Create figure
    fig = px.bar(df, x=['Unrelated revenues', 'Related revenues'], y=df.index,
                  orientation='h', title='Breakdown of revenue', text_auto='.0%')

    # Update layout settings
    fig.update_layout(
        xaxis=dict(
            title=None,
            tickformat='.0%'
        ),
        yaxis_title=None,
        legend=dict(
            title=dict(text=''),
            orientation='h',
            y=1.1
        ),
        plot_bgcolor='white',
        width=800,
        height=480
    )

    # Define position of text values
    for col in ['Unrelated revenues', 'Related revenues']:
        
        values_positions = ['outside' if value <= 0.05 else 'inside' for value in df[col]]
    
        fig.update_traces(
            textangle=0,
            textposition=values_positions,
            selector=dict(name=col)
        )

    # Add annotation if no values are availables (no bar displayed)
    for i, index in enumerate(df.index):
        if df.loc[index].isna().all():
            fig.add_annotation(
                    x=0.5,
                    y=df.index[i],
                    text='Information not provided by the company',
                    showarrow=False,
                    font=dict(color='black', size=13)
                )
    
    # Define style of hover on bars
    fig.update_traces(hovertemplate='%{y}: %{x}')

    fig.show()

In [49]:
display_related_and_unrelated_revenues_breakdown(df=df, company=company, year=year)

## #21 - evolution of tax havens use over time : % profit vs % employees in TH over time

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|line chart ?|||`company explo` need a filter by company||

In [63]:
def compute_tax_havens_use_evolution(df: pd.DataFrame, company: str) -> dict:
    """Compute the evolution of tax havens use by company over time.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name

    Returns:
        dict: tax havens percentage of profits and employees for each year.
    """
    
    # Filter rows with selected company and subset with necessary features
    features = ['jur_code', 'year', 'jur_tax_haven', 'profit_before_tax', 'employees']
    df = df.loc[(df['mnc'] == company), features]
    
    # Keep jurisdictions with profitable or missing revenues
    df = df.loc[(df['profit_before_tax'] >= 0) | (df['profit_before_tax'].isna())]

    # For all sum calculations below :
    # - Result NA : all jurisdictions values were NA ;
    # - Result 0 : at least one jurisdiction was reported as 0.

    # Calculate total profit and employees by year and tax haven status
    df = df.groupby(['year', 'jur_tax_haven'], as_index=False)[['profit_before_tax', 'employees']].sum(min_count=1)
    
    # Calculate total profits and employees for each year
    for year in df['year'].unique():
        df.loc[df['year'] == year, 'total_profit'] = df.loc[df['year'] == year, 'profit_before_tax'].sum(min_count=1)
        df.loc[df['year'] == year, 'total_employees'] = df.loc[df['year'] == year, 'employees'].sum(min_count=1)

    # Remove non tax haven jurisdictions
    df = df.loc[df['jur_tax_haven'] == True].reset_index()
    
    # Calculate percentages
    df['tax_havens_profit_%'] = df['profit_before_tax'] / df['total_profit']
    df['tax_havens_employees_%'] = df['employees'] / df['total_employees'] 

    # Convert necessary data to dictionnary
    data = df[['year', 'tax_havens_profit_%', 'tax_havens_employees_%']].to_dict()

    return data


def display_tax_havens_use_evolution(df: pd.DataFrame, company: str):
    """Display the evolution of tax havens use by company over time.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
    """
    
    # Compute data
    data = compute_tax_havens_use_evolution(df=df, company=company)

    # Create DataFrame
    df = pd.DataFrame.from_dict(data)

    # Rename columns
    df = df.rename(columns={
        'tax_havens_profit_%': 'Percentage of profits in tax havens', 
        'tax_havens_employees_%': 'Percentage of employees in tax havens'
    })
    
    # Create figure
    fig = px.bar(
        df,
        x='year',
        y=['Percentage of profits in tax havens', 'Percentage of employees in tax havens'], 
        barmode='group', 
        text_auto='.1%'
    )
    
    # Update layout settings
    fig.update_layout(
        title='Tax havens use in profitables jurisdictions',
        xaxis_title=None,
        yaxis_title=None,
        yaxis_tickformat ='.0%',
        legend=dict(
            title=dict(text=''),
            orientation='h'
        ),
        plot_bgcolor='white',
        width=800,
        height=480
    )
    
    fig.show()

In [64]:
print(company)
display_tax_havens_use_evolution(df=df, company=company)

RIO TINTO


## #22 - locations of profits booked vs. mean 3Y ETR

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|plot : x-axis : mean % profit, y-axis : mean ETR over 3Y<br>1 bubble per jurisdiction, with size of the bubble based on profit per employee in the JUR and a color code for tax havens vs others|by company, by JUR, calculate % total profit over several years, and tax rate (paid taxes/ pre-tax profit)||`company explo` need a filter by company||